# Generate patch.h for Dan's Chromcord Arduino
The first block is just setup, don't modify it (execute it once):

In [60]:
# Setup, don't modify this
import re
import collections
from pprint import pprint

patches = collections.OrderedDict()
Patches_Per_Show = 16 # so 1st knob is shows/stencil, 2nd is patch

def make():

    marker = "Generated by ipython patchGenerate"
    output = []
    output.append("// %s:\n" % marker)

    patch_names = []
    for show_name,show_patches in patches.iteritems():
        for patch_name,patch in show_patches.iteritems():
            full_name, constructed_lines = construct(show_name, patch_name, patch)
            patch_names.append(full_name)
            output.extend(constructed_lines)
        # fill in empties so we have 16/show
        for i in range(len(show_patches),Patches_Per_Show-1):
                patch_names.append('empty_patch')
    output.append("const byte** patches[] = {"+", ".join(patch_names)+"};")
    output.append('')
    
    for i,name in enumerate(patch_names):
        short_name = re.sub('^patch_', '', name)
        short_name = re.sub('_+','.', short_name)
        output.append( "prog_char patch_name_%s[] PROGMEM = \"%s\";" % (i,short_name))
    output.append(
        "PROGMEM const char *patch_names[] = {" +
        ", ".join(("patch_name_%s" % x) for x in range(0,len(patch_names))) +
        "};"
        )
    output.append('')
    
    output.append( 'const byte Patch_Count = count_of(patches);' )
    output.append("// End %s" % marker)
    
    print "\n".join(output)

def construct(show_name, patch_name, patch_list):
    fullname = "patch_%s_%s" % (show_name,patch_name)
    fullname = re.sub('[^a-z0-9_]', '', fullname, flags=re.IGNORECASE)
    
    if len(patch_list) != 4:
        raise Exception("Error, expected 4 zones, found %s for %s" %(len(patch_list),fullname))
    
    rez = []
    rez.append("// Patch build from %s" % fullname)
    patchdecl = []
    for i,zone_values in enumerate(patch_list):
        zonedef = "";
        zonedef_name = "%s_z%s" % (fullname,i);
        # print "Zone %s" % zonedef_name
        # pprint(zone_values)
        if (zone_values and len(zone_values)>0):
            zonedef += "  const byte %s[] = " % zonedef_name
            zonedef += "{"+ ",".join((str(x) for x in sorted(zone_values)))+", -1};"
            rez.append(zonedef)
        else:
            zonedef_name = 'patch_empty_zone'
        patchdecl.append(zonedef_name)
    
    rez.append("const byte* "+fullname+"[Zone_Count] = {"+", ".join(patchdecl)+"};")
    rez.append("")
    return [fullname, rez]
 

## Patches
Patches are structured as:
1. Group Name, like "Show 1"
1. Patch name, Like "Squarish #2"
1. A list of the Zones (rgb controllers), 1..4
1. Each lunit that belongs to the zone

E.g.:
```
patches['Show 1'] = {
    'squarish #2' : [ [0,1,2,3,4,5], [6,7,8], range(9,16), [] ],
    'center' : [ [0,1,2,3,5,8,9,11,12,13,14,15], [4,7,10], [], [] ],
    }
```

Keeps the order you specify.

In [61]:
# Edit, and Run to check
patches['Show 1'] = collections.OrderedDict([
    ['squarish #2', [ [0,1,2,3,4,5], [6,7,8], range(9,16), [] ]],
    ['center', [ [0,1,2,3,5,8,9,11,12,13,14,15], [4,7,10], [], [] ]],
    ])
patches['Show 2'] = collections.OrderedDict([
        ['cross', [ [0,1,2,3,5,8,9,11,12,13,14,15], [4,7,10], [], [] ]]
    ])
print "Reformatted:"
pprint( patches)


Reformatted:
OrderedDict([('Show 1', OrderedDict([('squarish #2', [[0, 1, 2, 3, 4, 5], [6, 7, 8], [9, 10, 11, 12, 13, 14, 15], []]), ('center', [[0, 1, 2, 3, 5, 8, 9, 11, 12, 13, 14, 15], [4, 7, 10], [], []])])), ('Show 2', OrderedDict([('cross', [[0, 1, 2, 3, 5, 8, 9, 11, 12, 13, 14, 15], [4, 7, 10], [], []])]))])


## Run this to make the C++ code:

In [62]:
# Run this and paste result into patches.h
make()

// Generated by ipython patchGenerate:

// Patch build from patch_Show1_squarish2
  const byte patch_Show1_squarish2_z0[] = {0,1,2,3,4,5, -1};
  const byte patch_Show1_squarish2_z1[] = {6,7,8, -1};
  const byte patch_Show1_squarish2_z2[] = {9,10,11,12,13,14,15, -1};
const byte* patch_Show1_squarish2[Zone_Count] = {patch_Show1_squarish2_z0, patch_Show1_squarish2_z1, patch_Show1_squarish2_z2, patch_empty_zone};

// Patch build from patch_Show1_center
  const byte patch_Show1_center_z0[] = {0,1,2,3,5,8,9,11,12,13,14,15, -1};
  const byte patch_Show1_center_z1[] = {4,7,10, -1};
const byte* patch_Show1_center[Zone_Count] = {patch_Show1_center_z0, patch_Show1_center_z1, patch_empty_zone, patch_empty_zone};

// Patch build from patch_Show2_cross
  const byte patch_Show2_cross_z0[] = {0,1,2,3,5,8,9,11,12,13,14,15, -1};
  const byte patch_Show2_cross_z1[] = {4,7,10, -1};
const byte* patch_Show2_cross[Zone_Count] = {patch_Show2_cross_z0, patch_Show2_cross_z1, patch_empty_zone, patch_empty_zone};

## And Paste 
the above result into patches.h, replacing the section:
```
// Generated by ....
...
// End Generatted by ...
```